Импорт библиотек

In [1]:
from selenium import webdriver
import time
import datetime
import math
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import openpyxl
import pandas as pd
import warnings

Функция для определения типа подвесок

In [2]:
def charms (i):
    name = str(result.iloc[i]['name'])
    if "крест" in name or "христиан" in name or "икона" in name or "мусульман" in name or "спаси" in name or "сохрани" in name or "месяц" in name or "аллах" in name or "пророк" in name or "будда" in name or "отче" in name or "молитва" in name or "икона" in name or "лунниц" in name or "ангел" in name or "эрцгамм" in name or "чудо" in name or "фатим" in name or "иисус" in name or "б.м" in name or "давид" in name or "матрон" in name or "госп" in name or "хамс" in name:
        return "КУЛЬТ"
    else:
        return "ДЕКОР"

Попытка вытащить вес из карточки по браслетам и цепям

In [3]:
def bracs (link):
    url =f'https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1257786&spp=30&nm={link}'
    sizes = requests.get(url, verify=False, timeout=30).json()
    try:
        r = sizes['data']['products'][0]['sizes'][0]['name']
        if r.rfind("-") > -1:
            wei = r[r.rfind("-")+1:].strip().lower().replace(",",".").replace("гр.","").replace("гр","")
        elif r.rfind("~") > -1:
            wei = r[r.rfind("~")+1:].strip().lower().replace(",", ".").replace("гр.","").replace("гр","")
        elif r.rfind(" ") > -1:
            wei = r[r.rfind(" ")+1:].strip().lower().replace(",", ".").replace("гр.","").replace("гр","")
        wei = float(wei)
        if wei > 0 and wei < 15:
            return wei
    except:
        return np.nan

In [4]:
married = True
parsesize = False
warnings.filterwarnings('ignore')
links=list()
spis = [143, 287, 431, 719, 1007, 1061, 1115, 1169, 1313, 1601, 1655, 1919, 2045, 2189,2405]
p=1
arts = list()
prices = list()
po = list()
date = list()
discount = list()
url = str(input('Введите ссылку:'))
url1 = url[:url.find("page=")+5]
url2 = url[url.find("&", url.find("page=")):]

Цикл пагинатор

In [5]:
mwd = webdriver.Chrome()
mwd.maximize_window()
while True:
    url = url1 + str(p) + url2
    mwd.get(url)
    time.sleep(1.5)
    step = 1900
    while True:
        step0=step-1900
        lh = mwd.execute_script("return document.body.scrollHeight")
        mwd.execute_script(f"window.scrollTo({step0}, {step});")
        time.sleep(1)
        nh = mwd.execute_script("return document.body.scrollHeight")
        step+=1900
        if nh==lh:
            break
    soup = bs(mwd.page_source, "html.parser")
    artss =  soup.find_all('article',id=True)
    for art in artss:
        arts.append(art['data-nm-id'])
    for art in artss:
        if str(art).find('price__lower-price wallet-price')>0:
            prl = art.findNext('ins',attrs={'class':'price__lower-price wallet-price'}).text.strip()[:-2]
            if prl.find('\xa0')>0:
                prl = prl.replace('\xa0','')
            prices.append(prl)
            po.append(math.ceil(int(prl)/0.95))
            discount.append(0.95)
        else:
            prl = art.findNext('ins',attrs={'class': 'price__lower-price'}).text.strip()[:-2]
            if prl.find('\xa0')>0:
                prl = prl.replace('\xa0','')
            prices.append(prl)
            po.append(prl)
            discount.append(1)
    if p == 1:
        page_counts = int(soup.find(attrs={'data-link': 'html{spaceFormatted:pagerModel.totalItems}'}).text.replace(' ',''))//100+1
    if p == page_counts:
        break
    p+=1
mwd.close()

In [6]:
weights = list()
inserts = list()
metals = list()
locks = list()
competitors = list()
names = list()
groups =list()

In [8]:
arts

['17641482',
 '35443032',
 '113256091',
 '139434433',
 '140882886',
 '13140596',
 '35495746',
 '7058175',
 '143294405',
 '37695907',
 '140993119',
 '139388358',
 '165282599',
 '152931636',
 '13140637',
 '13140601',
 '17641481',
 '13140598',
 '15877256',
 '163807314',
 '154024696',
 '198341800',
 '134882081',
 '149881588',
 '13140763',
 '140996083',
 '194274972',
 '155146930',
 '147953592',
 '15939685',
 '138797947',
 '116340526',
 '166064935',
 '13140597',
 '138793369',
 '10195507',
 '166059903',
 '158369105',
 '40021689',
 '176774563',
 '13140642',
 '54005592',
 '37047034',
 '13140618',
 '2306709',
 '194275145',
 '139385395',
 '101310307',
 '134162717',
 '159549142',
 '116340510',
 '37695906',
 '15877259',
 '13140773',
 '7567655',
 '10589462',
 '19939668',
 '72528431',
 '42933828',
 '66473290',
 '2943722',
 '3797193',
 '2890998',
 '13514451',
 '2776956',
 '43773177',
 '175090521',
 '9114223',
 '114502979',
 '117789961',
 '155960278',
 '13294573',
 '36212389',
 '7738746',
 '211494283',

Собираем инфу с карточки товара

In [7]:
for link in tqdm(arts):
    group = np.nan
    name = np.nan
    weight = np.nan
    lock_type = np.nan
    metal = np.nan
    insert = np.nan
    competitor = np.nan
    i = 0
    if int(link)//100000>spis[-1]:
        i = len(spis)
    else:
        while int(link)//100000 > spis[i]:
            i += 1
    i += 1
    if i < 10:
        i = "0"+str(i)
    current_link = f'https://basket-{i}.wbbasket.ru/vol{int(link)//100000}/part{int(link)//1000}/{link}/info/ru/card.json'
    try:
        basket = requests.get(current_link, verify=False, timeout=30).json()
    except:
        time.sleep(60)
        basket = requests.get(current_link, verify=False, timeout=30).json()
    try:
        competitor = basket['selling']['brand_name']
    except:
        pass
    try:
        name = basket['imt_name'].lower()
    except:
        pass
    try:
        match basket['subj_name']:
            case "Ювелирные браслеты": group = "ЦБ БРАСЛЕТЫ"
            case "Ювелирные цепочки": group = "ЦБ ЦЕПИ"
            case "Ювелирные кольца": group = "КОЛЬЦА"
            case "Ювелирные подвески": group = "ПОДВЕС"
            case "Ювелирные серьги": group = "СЕРЬГИ"
    except:
        pass
    try:
        for option in basket['options']:
            if option['name'] == 'Минимальный вес (г)':
                weight = float(option['value'].replace('г',''))
            elif option['name'] == 'Вставка':
                insert = option['value']
            elif option['name'] == 'Вид замка':
                lock_type = option['value']
            elif option['name'] == 'Состав ювелирного изделия':
                metal = option['value']
    except:
        pass
    try:
        for option in basket['grouped_options'][0]['options']:
            if option['name'] == 'Минимальный вес (г)':
                weight = float(option['value'].replace('г',''))
            elif option['name'] == 'Вставка':
                insert = option['value']
            elif option['name'] == 'Вид замка':
                lock_type = option['value']
            elif option['name'] == 'Состав ювелирного изделия':
                metal = option['value'].lower()
    except:
        pass
    try:
        if np.isnan(weight) and group in ["ЦБ БРАСЛЕТЫ","ЦБ ЦЕПИ"]:
            weight = float(bracs(link))
    except:
        pass
    try:
        if group == "ЦБ ЦЕПИ":
            for option in basket['options']:
                if 'плетен' in option['name']:
                    lock_type = option['value']
    except:
        pass
    try:
        if group == "ЦБ ЦЕПИ":
            for option in basket['grouped_options'][0]['options']:
                if 'плетен' in option['name']:
                    lock_type = option['value']
    except:
        pass
    date.append(datetime.date.today())
    weights.append(weight)
    inserts.append(insert)
    metals.append(metal)
    locks.append(lock_type)
    competitors.append(competitor)
    names.append(name)
    groups.append(group)

  0%|          | 0/5406 [01:00<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Условие для парсинга размеров

In [56]:
if group == "ЦБ ЦЕПИ" or group =="ЦБ БРАСЛЕТЫ" or group == "КОЛЬЦА":
    strsize = input('Вас интересует размер? [y/n]')
    if strsize.lower() in ['yes','y','д','да']:
        parsesize = True

Парсинг размеров

In [57]:
if parsesize:
    artsizes = list()
    for link in tqdm(arts):
        artsize = list()
        url =f'https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1257786&spp=30&nm={link}'
        try:
            sizes = requests.get(url, verify=False, timeout=30).json()
        except:
            artsizes.append(np.nan)
        try:
            for option in sizes['data']['products'][0]['sizes']:
                if len(option['stocks'])>0:
                    size = option['origName']
                    or1 = float(size.replace(",",".").replace("+",""))
                    if (or1 > 10) and (or1 < 100): 
                        artsize.append(or1)
                    try:
                        or1 = size[:size.find("-")]
                        or2 = size[size.find("-")+1:]
                        if or1< 100 and or2 <100 and or1>10 and or2>10:
                            for i in range(or1,or2+1):
                                artsize.append(i)
                    except:
                        pass
        except:
            pass
        artsizes.append(artsize)

Проверка на тип колец

In [59]:
if group == "КОЛЬЦА":
    mar = str(input('Вас интересуют обручальные кольца? [y/n]'))
    if len(mar.lower()) > 0 and mar not in ['да','yes','д','y']:
        married = False

Данные в датасет

In [60]:
if parsesize:
    result = pd.DataFrame({'data': date,
                       'competitor': competitors,
                       'name': names,
                       'group': groups,
                       'weight': weights,
                       'price': prices,
                       'price_old': po,
                       'discount': discount,
                       'insert': inserts,
                       'metal': metals,
                       'lock': locks,
                       'article': arts,
                       'sizes': artsizes})
else:
    result = pd.DataFrame({'data': date,
                       'competitor': competitors,
                       'name': names,
                       'group': groups,
                       'weight': weights,
                       'price': prices,
                       'price_old': po,
                       'discount': discount,
                       'insert': inserts,
                       'metal': metals,
                       'lock': locks,
                       'article': arts,})

In [61]:
result['price'] = result['price'].astype('int')
print(result.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   data        5000 non-null   object 
 1   competitor  4822 non-null   object 
 2   name        4998 non-null   object 
 3   group       5000 non-null   object 
 4   weight      4722 non-null   float64
 5   price       5000 non-null   int32  
 6   price_old   5000 non-null   object 
 7   discount    5000 non-null   float64
 8   insert      4935 non-null   object 
 9   metal       4989 non-null   object 
 10  lock        0 non-null      float64
 11  article     5000 non-null   object 
dtypes: float64(3), int32(1), object(8)
memory usage: 449.3+ KB
None


Работа с данными

In [62]:
groupdf = list()
ii = list()
avprice = list()
insertgroup = list()
charmgr = list()
result = result.drop(result["каучук" in result['insert'] or "жемчуг" in result['insert'] or "серебро" in result['metal'] or "каучук" in result['metal'] or "жемчуг" in result['metal'] or np.isnan(result['weight'])].index)
avgpr = result['price']/result['weight']
result.insert(loc=len(result.columns),column='Price per gramm',value=avgpr)
result = result.drop(result[result['Price per gramm']<3000].index)
cat = np.where(result['Price per gramm']<=5500, "до 5500",
        np.where(
        (result['Price per gramm'] <= 6000), '5 500 руб. - 6 000 руб.',
        np.where(
        (result['Price per gramm'] <= 6500), '6 000 руб. - 6 500 руб.',
        np.where(
        (result['Price per gramm'] <= 7000), '6 500 руб. - 7 000 руб.',
        np.where(
        (result['Price per gramm'] <= 8000), '7 000 руб. - 8 000 руб.',
        np.where(
        (result['Price per gramm'] <= 9000), '8 000 руб. - 9 000 руб.',
        np.where(
        (result['Price per gramm'] <= 10000), '9 000 руб. -10 000 руб.',  'свыше 10 000 руб.'
        )))))))
result.insert(loc=len(result.columns),column='group_price',value=cat)

Определяемся с категорией товарам и направлением

In [63]:
for i in tqdm(range(0,len(result))):
    insertgroup.append("")
    inss = str(result.iloc[i]['insert']).split(';')
    for ins in inss:
        ins = str(ins).strip().lower()
        if ins == "бриллиант" or ins == "сапфир" or ins =="рубин" or ins == "изумруд" or ins == "бриллиант натуральный" or ins == "сапфир натуральный" or ins == "рубин натуральный" or ins == "изумруд натуральный" or ins =="бриллианты" or ins == "":
            insertgroup[-1] = "ДК"
            charmgr.append('ДК')
            break
        elif ins == "фианит" or ins == "бриллиант искусственный" or ins == "бриллиант выращенный" or ins =="бриллианит" or ins == "бриллиант синтетический" or "фианит" in str(result.iloc[i]['insert']).lower():
            insertgroup[-1] = "ИФ"
        elif "нет" in ins or "без" in ins or  str(result.iloc[i]['insert']).lower().strip() == "золото" or str(result.iloc[i]['insert']).lower() == "эмаль" or ins == "nan":
            insertgroup[-1] = "БК"
            charmgr.append('БК')
            break
        elif "бр" in ins and "искусственный" not in str(result.iloc[i]['insert']).lower() and "выращенный" not in str(result.iloc[i]['insert']).lower() and "синтетический" not in str(result.iloc[i]['insert']).lower():
            insertgroup[-1] = "ДК"
            charmgr.append('ДК')
            break
        elif insertgroup[-1] != "ИФ":
            insertgroup[-1] = "ПДК"
    if insertgroup[-1] in ['ИФ', 'ПДК']:
        charmgr.append('ПДК')
    if result.iloc[i]['group'] == "ПОДВЕС" and insertgroup[-1] in ['БК','ИФ']:
        insertgroup[-1] = insertgroup[-1] + " " + result.iloc[i]['group'] + " " + charms(i)
    elif result.iloc[i]['group'] == "ПОДВЕС":
        insertgroup[-1] = insertgroup[-1] + " " + result.iloc[i]['group']
    elif result.iloc[i]['group'] == "СЕРЬГИ":
        insertgroup[-1] = insertgroup[-1] + " " + result.iloc[i]['group']
    elif result.iloc[i]['group'] in ['ЦБ ЦЕПИ', 'ЦБ БРАСЛЕТЫ']:
        insertgroup[-1] = result.iloc[i]['group']
        charmgr[-1] = "ЦБ"
    elif result.iloc[i]['group'] == "КОЛЬЦА" and married:
        insertgroup[-1] = insertgroup[-1] + " " + result.iloc[i]['group'] + " " + "ОБРУЧ"
    elif result.iloc[i]['group'] == "КОЛЬЦА":
        insertgroup[-1] = insertgroup[-1] + " " + result.iloc[i]['group']
    else:
        ii.append(i)
        charmgr.pop()
if len(ii)>0:
    result= result.drop(labels=ii,axis=0)
result.insert(loc=len(result.columns),column='product_group(ТГ)',value=insertgroup)
result.insert(loc=len(result.columns),column='product_direction(ТН)',value=charmgr)

100%|██████████| 4418/4418 [00:02<00:00, 2184.24it/s]


In [64]:
result

,data,competitor,name,group,weight,price,price_old,discount,insert,metal,lock,article,Price per gramm,group_price,product_group(ТГ),product_direction(ТН)
0,2024-03-19,KARATOV,кольцо обручальное золото 585 пробы,КОЛЬЦА,1.60,9438,9935,0.95,без вставки,золото красное,NaN,17641482,5898.750000,5 500 руб. - 6 000 руб.,БК КОЛЬЦА ОБРУЧ,БК
1,2024-03-19,SOKOLOV,кольцо золотое 585 пробы с фианитом помолвочное,КОЛЬЦА,0.93,5684,5984,0.95,фианит,золото красное,NaN,113256091,6111.827957,6 000 руб. - 6 500 руб.,ИФ КОЛЬЦА ОБРУЧ,ПДК
2,2024-03-19,KARATOV,кольцо обручальное 585 пробы,КОЛЬЦА,0.56,3590,3779,0.95,без вставки,золото красное,NaN,17641481,6410.714286,6 000 руб. - 6 500 руб.,БК КОЛЬЦА ОБРУЧ,БК
3,2024-03-19,REDzoloto,кольцо золотое 585 пробы с фианитами,КОЛЬЦА,1.33,12118,12756,0.95,фианит,золото красное,NaN,35443032,9111.278195,9 000 руб. -10 000 руб.,ИФ КОЛЬЦА ОБРУЧ,ПДК
4,2024-03-19,Ювелирный магазин ЗОЛОТОЕ ДЕРЕВО,золотое кольцо обручальное,КОЛЬЦА,1.70,9851,10370,0.95,без вставки,золото красное,NaN,140882886,5794.705882,5 500 руб. - 6 000 руб.,БК КОЛЬЦА ОБРУЧ,БК
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-03-19,Maxim Demidov,из белого золота (585 пробы) с натуральным тан...,КОЛЬЦА,4.22,275330,275330,1.00,танзанит натуральный,золото белое; золото радированное,NaN,187217319,65244.075829,свыше 10 000 руб.,ПДК КОЛЬЦА ОБРУЧ,ПДК
4996,2024-03-19,Maxim Demidov,из белого золота (585 пробы) с натуральным тан...,КОЛЬЦА,4.31,246081,246081,1.00,танзанит натуральный,золото белое,NaN,187217294,57095.359629,свыше 10 000 руб.,ПДК КОЛЬЦА ОБРУЧ,ПДК
4997,2024-03-19,Maxim Demidov,из белого золота (585 пробы) с натуральным тур...,КОЛЬЦА,7.14,239254,239254,1.00,турмалин натуральный,золото белое; золото радированное,NaN,187217316,33508.963585,свыше 10 000 руб.,ПДК КОЛЬЦА ОБРУЧ,ПДК
4998,2024-03-19,Maxim Demidov,из белого золота (585 пробы) с натуральным сап...,КОЛЬЦА,4.23,209144,209144,1.00,сапфир натуральный,золото белое; золото радированное,NaN,187217338,49443.026005,свыше 10 000 руб.,ДК КОЛЬЦА ОБРУЧ,ДК


Расписываем серьги 

In [65]:
for i in tqdm(result.index):
    if result.at[i, 'group'] == "СЕРЬГИ":
        if "англ" in str(result.at[i, 'lock']).lower() or "нгл" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "английский"
        elif "франц" in str(result.at[i, 'lock']).lower() or "зск" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "французский"
        elif "штиф" in str(result.at[i, 'lock']).lower() or "фтов" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "штифтовой"
        elif "конг" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "конго"
        elif "винт" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "винтовой"
        elif "зажим" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "зажим"
        elif "петля" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "петля"
        elif "прот" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "протяжки"
        elif "пус" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "пуссеты"
        elif "прод" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "продевки"
        elif "без" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "без застежки"
        else:
            result.at[i, 'lock'] = "неопределенно"

100%|██████████| 4418/4418 [00:00<00:00, 220771.25it/s]


Расписываем цепи

In [66]:
for i in tqdm(result.index):
    if result.at[i, 'group'] == "ЦБ ЦЕПИ":
        if "ром" in str(result.at[i, 'lock']).lower() and "двой" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "ромб двойной"
        elif "ром" in str(result.at[i, 'lock']).lower() and "трой" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "ромб тройной"
        elif "ром" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "ромб"
        elif "бис" in str(result.at[i, 'lock']).lower() or "марк" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "бисмарк"
        elif "нон" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "нонна"
        elif "якор" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "якорь"
        elif "панц" in str(result.at[i, 'lock']).lower() or "цирь" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "панцирь"
        elif "синг" in str(result.at[i, 'lock']).lower() or "апур" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "сингапур"
        elif "лав" in str(result.at[i, 'lock']).lower() or "лов" in str(result.at[i, 'lock']).lower() or "lov" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "лав"
        elif "тонд" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "тондо"
        elif "фанта" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "фантазия"
        elif "рол" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "ролло"
        elif "рол" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "ролло"
        elif "фига" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "фигаро"
        elif "корд" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "корда"
        elif "улит" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "улитка"
        elif "перл" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "перлина"
        elif "сней" in str(result.at[i, 'lock']).lower() or "snak" in str(result.at[i, 'lock']).lower() or "змей" in str(result.at[i, 'lock']).lower():
            result.at[i, 'lock'] = "снейк"
        else:
            result.at[i, 'lock'] = "неопределенно"

100%|██████████| 4418/4418 [00:00<00:00, 104204.85it/s]


Данные в файл на сервер и к себе

In [67]:
# if parsesize:
#   adres = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\tg_tn'.encode().decode('UTF-8')+r'\wildberries_' + str(datetime.date.today().    day) + "_" + str(datetime.date.today().month) + "_" + str(datetime.date.today().year)+"_with_size"+".xlsx"
# else:
#     adres = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\tg_tn'.encode().decode('UTF-8')+r'\wildberries_' + str(datetime.date.today().    day) + "_" + str(datetime.date.today().month) + "_" + str(datetime.date.today().year)+".xlsx"
# try:
#     oldres = pd.read_excel(adres)
# except:
#     oldres = pd.DataFrame()
# oldres = pd.concat([oldres,result],ignore_index=True)
# oldres.to_excel(adres, index=False)
if parsesize:
    adresm = "wildberries_"+ str(datetime.date.today().day) + "_" + str(datetime.date.today().month) + "_" + str(datetime.date.today().year)+"_with_size"+".xlsx"
else:
    adresm = "wildberries_"+ str(datetime.date.today().day) + "_" + str(datetime.date.today().month) + "_" + str(datetime.date.today().year)+".xlsx"
try:
    oldresm = pd.read_excel(adresm)
except:
    oldresm = pd.DataFrame()
oldresm = pd.concat([oldresm,result],ignore_index=True)
oldresm.to_excel(adresm, index=False)

In [68]:
res = oldresm[oldresm['group'] == "СЕРЬГИ"]
if len(res)>0:
    print(res['lock'].value_counts())

In [69]:
res = oldresm[oldresm['group'] == "ЦБ ЦЕПИ"]
if len(res)>0:
    print(res['lock'].value_counts())